In [1]:
import math
import random

# 定数の定義
d0 = 1.0  # 基準距離 [m]
PL_bar = 40.0  # 平均RSSI [dBm]
gamma = 3.0  # 距離減衰定数
sigma = 2.0  # X_sigma の標準偏差 [dB]

# ガウス分布に従うランダムな値 X_sigma を生成
X_sigma = random.gauss(0, sigma)

def calculate_distance(PL):
    """
    与えられた PL [dBm] の値から距離 d [m] を計算します。
    """
    exponent = (PL_bar - PL + X_sigma) / (10 * gamma)
    d = d0 * (10 ** exponent)
    return d

# 例として PL = 50 [dBm] の場合の距離を計算
PL_input = 50.0  # 与えられた RSSI 値 [dBm]
distance = calculate_distance(PL_input)

print(f"測定されたRSSIが {PL_input} [dBm] のとき、距離 d は約 {distance:.2f} [m]")

測定されたRSSIが 50.0 [dBm] のとき、距離 d は約 0.37 [m]


In [2]:
import requests
import nest_asyncio
#import macaddress
# from mac_vendor_lookup import MacLookup
from manuf import manuf

# Initialize the Manuf instance
p = manuf.MacParser()
# Allow nested event loops
nest_asyncio.apply()

# mac = MacLookup()
# mac.update_vendors() 



def get_vendor_from_mac(mac):
    """
    MACアドレスからベンダ情報を取得します。
    """
    vendor = p.get_manuf(mac)
    return vendor if vendor else "Unknown Vendor"
# 例としてMACアドレスを指定して検索
mac_address = "44:38:39:ff:ef:57"
vendor = get_vendor_from_mac(mac_address)
print(f"The vendor for MAC address {mac_address} is: {vendor}")


The vendor for MAC address 44:38:39:ff:ef:57 is: CumulusN


In [1]:
# output.txtの結果を読み込む
# timestamp, MACアドレス, RSSIがTabで仕切られて一行にまとまっているので、これを分ける
with open("output.txt", "r") as f:
    lines = f.readlines()

# MACアドレスごとのRSSIを計算
mac_rssi = {}
# MACアドレスごとの検出された数
mac_count = {}
# MACアドレスごとの最後に検出された時間
mac_last_timestamp = {}
# MACアドレスごとのベンダー名
mac_vendor = {}


import re

# BLEフォーマットに一致するパターン
pattern = re.compile(r'^\d+\.\d+\t([0-9a-fA-F]{2}:){5}[0-9a-fA-F]{2}\t-?\d+\tBLE$')


for line in lines:
    # 形式に合わない行をスキップ
    if pattern.match(line):
        continue
    timestamp, mac, rssi, signaltype= line.strip().split("\t")
    rssi = float(rssi)

    if mac not in mac_rssi:
        mac_rssi[mac] = []

    if mac not in mac_last_timestamp:
        mac_last_timestamp[mac] = 0

    mac_last_timestamp[mac] = max(mac_last_timestamp[mac], float(timestamp))
    mac_rssi[mac].append(rssi)


    # MACアドレスごとのベンダー名を取得
    if mac not in mac_vendor:
        # mac_vendor[mac] = get_vendor_from_mac(mac)
        pass

# MACアドレスごとのRSSIの検出数を計算
for mac, rssi_list in mac_rssi.items():
    mac_count[mac] = len(rssi_list)

# MACアドレスを検出された回数の多い順にソート
mac_count = dict(sorted(mac_count.items(), key=lambda x: x[1], reverse=True))

# MACアドレスごとの平均RSSIを計算
mac_avg_rssi = {}
for mac, rssi_list in mac_rssi.items():
    avg_rssi = sum(rssi_list) / len(rssi_list)
    mac_avg_rssi[mac] = avg_rssi

print("MACアドレスごとの平均RSSI")
for mac in mac_count.keys():
    # print(f"{mac}: {mac_avg_rssi[mac]:.2f} [dBm] ({mac_count[mac]}回検出)" + f" ({mac_vendor[mac]})")
    print(f"{mac}: {mac_avg_rssi[mac]:.2f} [dBm] ({mac_count[mac]}回検出)")    

ValueError: too many values to unpack (expected 4)